### AWS Conntrol Tower
##### Periodic Daily Config Recordings

In [ ]:
import boto3
import botoplus.botoplus as botoplus

sessions = botoplus.default()
accounts = botoplus.accounts()

home = 'us-east-2' # Define Management Region

for account in accounts:

    if account['awsalias'] != 'Portal': # Exclude Management Account

        print(str(account['awsaccount'])+' '+account['awsalias'])

        sts = sessions.client('sts')

        assumed = sts.assume_role(
            RoleArn = 'arn:aws:iam::'+account['awsaccount']+':role/AWSControlTowerExecution',
            RoleSessionName = 'ControlTowerBreakGlass'
        )
        session = boto3.Session(
            aws_access_key_id = assumed['Credentials']['AccessKeyId'],
            aws_secret_access_key = assumed['Credentials']['SecretAccessKey'],
            aws_session_token = assumed['Credentials']['SessionToken']
        )

        ec2 = session.client('ec2', region_name=home)
        regions = ec2.describe_regions()
     
        for region in regions['Regions']:

            print(' - '+region['RegionName'])

            sts = sessions.client('sts', region_name=region['RegionName'])

            assumed = sts.assume_role(
                RoleArn = 'arn:aws:iam::'+account['awsaccount']+':role/AWSControlTowerExecution',
                RoleSessionName = 'CTBreakGlass-'+region['RegionName']
            )
            session = boto3.Session(
                aws_access_key_id = assumed['Credentials']['AccessKeyId'],
                aws_secret_access_key = assumed['Credentials']['SecretAccessKey'],
                aws_session_token = assumed['Credentials']['SessionToken']
            )

            config = session.client('config', region_name=region['RegionName'])
            recorders = config.describe_configuration_recorders()

            if len(recorders['ConfigurationRecorders']) != 0:
                if region['RegionName'] != home:
                    for record in recorders['ConfigurationRecorders']:
                        config.put_configuration_recorder(
                            ConfigurationRecorder={
                                'name': record['name'],
                                'roleARN': record['roleARN'],
                                'recordingGroup': {
                                    'allSupported': True,
                                    'includeGlobalResourceTypes': False,
                                    'recordingStrategy': {
                                        'useOnly': 'ALL_SUPPORTED_RESOURCE_TYPES'
                                    }
                                },
                                'recordingMode': {
                                    'recordingFrequency': 'DAILY'
                                }
                            }
                        )
                else:
                    for record in recorders['ConfigurationRecorders']:
                        config.put_configuration_recorder(
                            ConfigurationRecorder={
                                'name': record['name'],
                                'roleARN': record['roleARN'],
                                'recordingGroup': {
                                    'allSupported': True,
                                    'includeGlobalResourceTypes': True,
                                    'recordingStrategy': {
                                        'useOnly': 'ALL_SUPPORTED_RESOURCE_TYPES'
                                    }
                                },
                                'recordingMode': {
                                    'recordingFrequency': 'DAILY'
                                }
                            }
                        )

##### Validate Daily Config Recordings

In [ ]:
import botoplus.botoplus as botoplus

accounts = botoplus.accounts()

for account in accounts:

    print(str(account['awsaccount'])+' '+account['awsalias'])
    session = botoplus.defaults(account['awsaccount'])
    ec2 = session.client('ec2')
    regions = ec2.describe_regions()

    for region in regions['Regions']:
        config = session.client('config', region_name = region['RegionName'])
        response = config.describe_configuration_recorders()
        for record in response['ConfigurationRecorders']:
            print(' - '+region['RegionName']+': '+record['name']+' '+record['recordingMode']['recordingFrequency'])